<table style="background-color: white; text-align: center"><tr>
<td style="background-color: white"> <img src="UL_logo.png" alt="Drawing" style="width: 100px;"/> </td>
<td style="background-color: white"> <img src="FCUL_logo.png" alt="Drawing" style="width: 200px;"/> </td>
</tr></table>

<p style="text-align: center; font-size: 20px;">Faculdade de Ciências da Universidade de Lisboa</p>

<br>
<br>
<br>

<p style="text-align: center; font-size: 18px; font-style: italic;">Projecto fase II</p>


<br>
<br>
<br>
<br>
<br>
<br>

<p style="text-align: center; font-size: 16px; font-style: italic;">Fundamentos da Web-semântica</p>

<p style="text-align: center; font-size: 16px; font-style: italic;">Docente: André Falcão</p>

<br>
<br>
<br>
<br>
<br>
<br>
<br>


<p style="text-align: right; font-size: 14px;">João Pinto Nº 55018</p>
<p style="text-align: right; font-size: 14px;">Ricardo Meireles Nº 55841</p>


<p style="text-align: center; font-size: 12px; font-weight: bold; font-style: italic;">Ano Lectivo 2020-2021</p>

-------------------------------------------------------------------------

Para esta fase de entrega do trabalho foram feitas as seguintes mudanças/funcionalidades:

<ol>
    <li>Revisão dos triplos após feedback do docente, nomeadamente:</li>
    <ul>
    <li>Separação do primeiro e último nome</li>
    <li>Separação dos anos jogados do formato 1984-1986 para addicionar os anos todos jogados eg: 1984,1985,1986</li>
    <li>Tratamento do player id para não conter o nome</li>
    <li>Tratamento do birth year e birth place</li>
    </ul>
    
<li>Ligaçao e utilizaçao do reasoner Fuseki para realizar as queries em vez de usar a biblioteca rdflib</li>
<ul>  
<li>Efectuamos a ligação das queries presentes neste notebook ao fuseki através da seguinte lógica:</li>
    <li>Primeiro, alteramos o fuseki para seguir as configurações presentes no ficheiro config_fuseki.ttl, aonde garantimos que este está a utilizar os dados gerados e tratados por este notebook, baseado dos data sets fornecidos.</li>
    <li>Por fim, listamos o reasoner presente ao Fuseki para apenas RDFS.</li>
</ul>

<li>Leitura e tratamento de um ficheiro com o nome, info, e as equipas para um melhor estruturamento da informação</li>
<li>Utilização do namespace Schema.org para uniformação da informaçao.</li>

<ul>   
<li>Nesta caso usamos o https://schema.org/Person , https://schema.org/athlete e https://schema.org/SportsTeam utilizando as suas proprieadades sempre que possivel (excepcão em termos muito especificos ao futebol Americano)
   nos triplos.</li>
   <li>Por exemplo usando o schema addicionamos os jogadores a equipa através da propriedade "athlete" do Sports Team alterando a forma como o estavamos a fazer anteriormente.</li>
</ul>
        
<li>Usando um SPARQLWrapper fazemos um query à DBPedia usando a propriedade rdfs:label = "nome de equipa" pedindo informação extra nomedamente:</li>
    <ul>
    <li>Treinador da equipa</li>
    <li>CEO da equipa</li>
    <li>Abstracto da equipa</li>
    <li>Número de "superbowls" conquistadas</li>
    <li>Após feito o query tratamos a informação recebida em JSON e acrescentamo-la atráves de triplos aos dados de cada equipa.</li> 
    </ul>
</ol>

In [ ]:
import csv
import rdflib
import re
from rdflib import Graph, Literal, Namespace, URIRef, BNode
from rdflib.plugins.sparql import prepareQuery
from rdflib.namespace import DCTERMS, RDF, RDFS, XSD
from urllib.parse import quote
from SPARQLWrapper import SPARQLWrapper, XML, TSV, POST, DIGEST, JSON
from string import Template


ListaEquipas = ["Equipa"]

NFL = Namespace('http://www.nfl.com/')
pff = Namespace('http://www.pff.com/')

Schema = Namespace("http://schema.org/")
Schema.Person = rdflib.term.URIRef(u'http://schema.org/Person')
Schema.City = rdflib.term.URIRef(u'http://schema.org/City')
Schema.SportsTeam = rdflib.term.URIRef(u'http://schema.org/SportsTeam')
Schema.Athlete = rdflib.term.URIRef(u'http://schema.org/athlete')
Schema.CollegeOrUniversity = rdflib.term.URIRef(u'http://schema.org/CollegeOrUniversity')

statid=1

dados_equipas = csv.DictReader(open("nfl_teams.csv"))
dados_base = csv.DictReader(open("Basic_Stats_Sample.csv"))
dados_passing = csv.DictReader(open("Career_Stats_Passing_Sample.csv"))


dados = Graph()

In [ ]:
for row in dados_equipas:
    idequipa =row['ID']
    nomeequipa = row ['Name']
    equipa=URIRef(NFL+idequipa)
    
    ListaEquipas.append(nomeequipa) #acrescentar o nome das equipas a uma lista para ser mais facil pesquisar

    dados.add((equipa, RDF.type, Schema.SportsTeam)) #tipo sportsteam baseado no: https://schema.org/SportsTeam
    dados.add((equipa, RDFS.label, Literal(nomeequipa,lang="en"))) #label com lingua EN para ser possivel pesquisar no dbpedia
    dados.add((equipa,Schema.name,Literal(nomeequipa))) #nome da equipa
    dados.add((equipa,Schema.sport,Literal("American Football"))) #indicar o desporto
    
    
    # query spark á dbpedia, pede o treinador, CEO, ano fundaçao, numero superbows e abstacto das equipas cujo label seja 
    #igual ao nome lido no ficheiro.
    
    sparql = SPARQLWrapper("http://dbpedia.org/sparql")
    sparql.addDefaultGraph("http://dbpedia.org")
    query =Template( """
    PREFIX dbpedia: <http://dbpedia.org/resource/> 
    Select ?team ?coach ?owner ?fd ?sb ?abs
     WHERE{ 
        ?team rdfs:label "$nome_equipa"@en;
        dbo:coach ?coach;
        dbo:owner ?owner;
        dbo:formationDate ?fd;
        dbp:noSbChamps ?sb;
        dbo:abstract ?abs.
        FILTER (lang(?abs) = 'en')
    }
        LIMIT 1
    """)


    sparql.setQuery(query.substitute(nome_equipa=nomeequipa))
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()

    #depois de feito os dados é preciso trata-los retirando os valores

    for result in results["results"]["bindings"]:
     if (str(result["coach"]["value"])):
      coach = str(result["coach"]["value"]) 
      owner = str(result["owner"]["value"])
      abs = str(result["abs"]["value"])
      fd = str(result["fd"]["value"])
      sb = int(result["sb"]["value"])   

    #Aqui acrescentamos os triplos usando o schema, por vezes (pensamos por falhas na ligaçao a dbpedia)
    # o query retorna apenas o cabeçalho e por isso é necessario o if para o codigo nao crashar.  
    if 'coach' in locals():
     dados.add((equipa,Schema.coach,URIRef(coach)))
     dados.add((equipa,Schema.owner,URIRef(owner)))
     dados.add((equipa,Schema.noSbChamps ,Literal(sb,datatype=XSD.int)))  
     dados.add((equipa,Schema.abstract  ,Literal(abs))) 
     dados.add((equipa,Schema.formationDate  ,Literal(fd,datatype=XSD.date))) 


In [ ]:
for row in dados_base:
    
    id =row['Player Id'] 

    current_team = row['Current Team']

    name = Literal(row['Name'])
    position = Literal(row['Position'])
    player_number = Literal(row['Number'],datatype=XSD.int)
    current_status = Literal(row['Current Status'])
    experience = Literal(row['Experience'])
    birth_place = row['Birth Place']
    birth_day = Literal(row['Birthday'],datatype=XSD.date)
    college = Literal(row['College'])
    high_school = Literal(row['High School'])
    height = Literal(row['Height (inches)'],datatype=XSD.int)
    weight = Literal(row['Weight (lbs)'],datatype=XSD.int)
    year_played = Literal(row['Years Played'])

    #tratamento do player para tirar o nome ao id e addicionar como person e atleta
    
    string_player = str(id)
    player = URIRef(NFL+string_player.split('/')[1])
  
    #Atleta é uma Pessoa
    
    dados.add((player, RDF.type, Schema.Athlete))
    dados.add((Schema.Athlete, RDFS.subClassOf, Schema.Person))
    
    dados.add((player, Schema.Gender,  URIRef("http://schema.org/Male") ))

    
    #addicionar birthday ao schema
    dados.add((player,Schema.birthDate,birth_day))

    # Tratamento do campo name, separar por nome e nome de familia e addicionar o triplo com o schema
    string_name = str(name)
    dados.add((player, Schema.givenName, Literal(string_name.split(',')[1] )))  
    dados.add((player, Schema.familyName, Literal(string_name.split(',')[0] )))  
    
    # Tratamento do years played, separar por temporada e addicionar todas ao triplo
    string_years = str(year_played)
    if  string_years:  
     nums = [int(n) for n in string_years.split('-')] 
    x = range(nums[0], nums[1]+1)
    for n in x: 
     dados.add((player, NFL.yearPlayed, Literal(n,datatype=XSD.int)))  

    # Addicionar jogador como atlete do schema Sports Team
    if current_team:  
      dados.add((URIRef(NFL+str(ListaEquipas.index(current_team))),Schema.atlete,player))  
    
    #tratar a experience e addicionar ao schema
    
    string_exp = row['Experience']
    if string_exp:
     temp = re.findall(r'\d+', string_exp) 
     res = list(map(int, temp)) 
     dados.add((player, NFL.experience, Literal(res[0],datatype=XSD.int)))
    
    #Tratamento do campo Birthplace, retirar estado e addicionar o triplo com o schema
    if birth_place: 
     string_birth = str(birth_place)
     dados.add((player, Schema.birthPlace, Literal(string_birth.split(',')[0] ))) 
        
    dados.add((player, NFL.currentStatus, current_status)) #addicionar current status
    
    dados.add((player, NFL.college, college)) #addicionar college
    
    #addicionar Hight and Weight
    dados.add((player, Schema.height, height))
    dados.add((player, Schema.weight, weight))
    
    if position:   #addicionar posiçao se existir no ficheiro
        dados.add((player, NFL.position, position))
    if player_number:  #addicionar numero do jogador se existir no ficheiro
        dados.add((player, NFL.playerNumber, player_number))
        
    if high_school: #tratamento de high school (retirar o hs) e addicionar ao triploe
        hs_string = high_school
        dados.add((player, NFL.highSchool, Literal(hs_string.split('HS')[0]))) 
        
    dados.add((player, RDFS.label, Literal(nomeequipa,lang="en")))

In [ ]:
for row in dados_passing:
      
     id =row['Player Id'] 
    
     player = URIRef(NFL+id)

     year = Literal(row['Year'])
     
     playerseason= URIRef(NFL+id+"/"+quote(row['Year']))
    
     name = URIRef(NFL+quote(row['Name']))
     team = URIRef(NFL+quote(row['Team']))
     games_played = Literal(row['Games Played'])
     passes_attempted = Literal(row['Passes Attempted'])
     passes_completed = Literal(row['Passes Completed'])
     completion_percentage = Literal(row['Completion Percentage'])
     pass_att_per_game = Literal(row['Pass Attempts Per Game'])
     passing_yards = Literal(row['Passing Yards'])
    
     passing_yards_att = Literal(row['Passing Yards Per Attempt'])
     passing_yards_game = Literal(row['Passing Yards Per Game'])
     td_passes = Literal(row['TD Passes'])
     percentage_tds_per_attemp = Literal(row['Percentage of TDs per Attempts'])
    
     interceptions = Literal(row['Ints'])
     interceptions_rate = Literal(row['Ints'])
     longest_pass = Literal(row['Longest Pass'])
     passes_longer_20yards = Literal(row['Passes Longer than 20 Yards'])
    
     passes_longer_40yards = Literal(row['Passes Longer than 40 Yards'])
     sacks = Literal(row['Sacks'])
     sacked_yards_lost = Literal(row['Sacked Yards Lost'])
     passer_rating = Literal(row['Passer Rating'])
  

     dados.add((playerseason, RDF.type, NFL.playerseason))
     dados.add((playerseason, RDFS.label, Literal(year, lang="en")))

     dados.add((player, NFL.playerseason, playerseason))

     dados.add((playerseason,NFL.team,team))
     dados.add((playerseason,NFL.games_played,games_played))
     dados.add((playerseason,NFL.passes_attempted,passes_attempted))
     dados.add((playerseason,NFL.passes_completed,passes_completed))
     dados.add((playerseason,NFL.completion_percentage,completion_percentage))
     dados.add((playerseason,NFL.pass_att_per_game,pass_att_per_game))
     dados.add((playerseason,NFL.passing_yards,passing_yards))
    
     dados.add((playerseason,NFL.passing_yards_att,passing_yards_att))
     dados.add((playerseason,NFL.passing_yards_game,passing_yards_game))
     dados.add((playerseason,NFL.td_passes,td_passes))
     dados.add((playerseason,NFL.percentage_tds_per_attemp,percentage_tds_per_attemp))
    
     dados.add((playerseason,NFL.interceptions,interceptions))
     dados.add((playerseason,NFL.interceptions_rate,interceptions_rate))
     dados.add((playerseason,NFL.longest_pass,longest_pass))
     dados.add((playerseason,NFL.passes_longer_20yards,passes_longer_20yards))
    
     dados.add((playerseason,NFL.passes_longer_40yards,passes_longer_40yards))
     dados.add((playerseason,NFL.sacks,sacks))
     dados.add((playerseason,NFL.sacked_yards_lost,sacked_yards_lost))
     dados.add((playerseason,NFL.passer_rating,passer_rating))
    
     dados.add((playerseason, NFL.name, name ))  
     dados.add((playerseason, NFL.year, year ))  

In [ ]:
for row in dados_passing:
    
     stat = URIRef(pff+str(statid))
     statid= statid+1
    
     player = URIRef(NFL+id)
     name = URIRef(NFL+quote(row['Name']))
     team = URIRef(NFL+quote(row['Team']))
     games_played = Literal(row['Games Played'])
     passes_attempted = Literal(row['Passes Attempted'])
     passes_completed = Literal(row['Passes Completed'])
     completion_percentage = Literal(row['Completion Percentage'])
     pass_att_per_game = Literal(row['Pass Attempts Per Game'])
     passing_yards = Literal(row['Passing Yards'])
    
     passing_yards_att = Literal(row['Passing Yards Per Attempt'])
     passing_yards_game = Literal(row['Passing Yards Per Game'])
     td_passes = Literal(row['TD Passes'])
     percentage_tds_per_attemp = Literal(row['Percentage of TDs per Attempts'])
    
     interceptions = Literal(row['Ints'])
     interceptions_rate = Literal(row['Ints'])
     longest_pass = Literal(row['Longest Pass'])
     passes_longer_20yards = Literal(row['Passes Longer than 20 Yards'])
    
     passes_longer_40yards = Literal(row['Passes Longer than 40 Yards'])
     sacks = Literal(row['Sacks'])
     sacked_yards_lost = Literal(row['Sacked Yards Lost'])
     passer_rating = Literal(row['Passer Rating'])
    
     dados.add((stat, RDF.type, pff.stat))
     dados.add((stat, NFL.name, name))
     dados.add((stat, NFL.player, player))
     dados.add((stat,pff.passing_yards,passing_yards))
    
     dados.add((stat,NFL.team,team))
     dados.add((stat,pff.games_played,games_played))
     dados.add((stat,pff.passes_attempted,passes_attempted))
     dados.add((stat,pff.passes_completed,passes_completed))
     dados.add((stat,pff.completion_percentage,completion_percentage))
     dados.add((stat,pff.pass_att_per_game,pass_att_per_game))
     dados.add((stat,pff.passing_yards,passing_yards))
    
     dados.add((stat,pff.passing_yards_att,passing_yards_att))
     dados.add((stat,pff.passing_yards_game,passing_yards_game))
     dados.add((stat,pff.td_passes,td_passes))
     dados.add((stat,pff.percentage_tds_per_attemp,percentage_tds_per_attemp))
    
     dados.add((stat,pff.interceptions,interceptions))
     dados.add((stat,pff.interceptions_rate,interceptions_rate))
     dados.add((stat,pff.longest_pass,longest_pass))
     dados.add((stat,pff.passes_longer_20yards,passes_longer_20yards))
    
     dados.add((stat,pff.passes_longer_40yards,passes_longer_40yards))
     dados.add((stat,pff.sacks,sacks))
     dados.add((stat,pff.sacked_yards_lost,sacked_yards_lost))
     dados.add((stat,pff.passer_rating,passer_rating))

In [ ]:
dados.bind("NFL",NFL)
dados.bind("Pro football Focus", pff)
dados.bind("Schema", Schema)

In [ ]:
dados_final = dados.serialize(format="turtle")

print(dados_final.decode("utf-8"))

In [ ]:
fil=open("nfl_data_v2.ttl", "wb")
fil.write(dados_final)
fil.close()

In [ ]:
# Execução de Queries no Fuseki por Inferência

# Pré-requisitos:
#
# Windows:
# Na pasta do fuseki em: \apache-jena-fuseki-3.16.0\fuseki-server.bat, alterar a execução para:
# java -Xmx1200M -jar fuseki-server.jar --conf config_fuseki.ttl %*

# No ficheiro config_fuseki.ttl, garantir que se encontram os seguintes parametros:
#
# :baseModel a ja:MemoryModel ;
#     ja:content [ja:externalContent <file:C:/path/to/file/nfl_data_v2.ttl> ] ;

# WARNING : O ficheiro nfl_data_v2.ttl tem de estar gerado pelo notebook!

# Mudar o módelo de inferência para apenas RDFS:

# :model_inf a ja:InfModel ;
#      ja:baseModel :baseModel ;
#      ja:reasoner [
#          ja:reasonerURL <http://jena.hpl.hp.com/2003/RDFSExptRuleReasoner>
#      ] .

In [ ]:
sparql = SPARQLWrapper("http://localhost:3030/dataset/update")
sparql.setMethod(POST)

query="""
prefix NFL: <http://www.nfl.com/>
prefix Schema: <http://schema.org/>
prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#>
prefix xsd: <http://www.w3.org/2001/XMLSchema#>

INSERT {?s a NFL:offensivePlayer . }
    WHERE {
    ?s NFL:position ?o .
    FILTER (?o IN ("C","OG","OT", "QB", "HB", "FB", "WR", "TE"))
    }
"""

sparql.setQuery(query)
results = sparql.query()

print(results)

In [ ]:
sparql = SPARQLWrapper("http://localhost:3030/dataset/update")
sparql.setMethod(POST)

query="""
prefix NFL: <http://www.nfl.com/>
prefix Schema: <http://schema.org/>
prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#>
prefix xsd: <http://www.w3.org/2001/XMLSchema#>

INSERT {?s a NFL:defensivePlayer . }
    WHERE {
    ?s NFL:position ?o .
    FILTER (?o IN ("DT", "DE", "MLB", "OLB", "CB", "S"))
    }
"""

sparql.setQuery(query)
results = sparql.query()

print(results)


In [ ]:
# Query 1 - Todas as pessoas existentes

from SPARQLWrapper import SPARQLWrapper, XML, TSV, POST, DIGEST

sparql = SPARQLWrapper(("http://localhost:3030/dataset/query")

query = """
prefix NFL: <http://www.nfl.com/>
prefix Schema: <http://schema.org/>
prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#>
prefix xsd: <http://www.w3.org/2001/XMLSchema#>

SELECT ?s ?p ?o 
WHERE {
  ?s ?p Schema:Person .
} 
"""
sparql.setQuery(query)
results = sparql.query()
g=results.convert()

print(g.serialize(format='ttl').decode("utf-8"))
                       
                       
                       
#dados.add((player, RDF.type, Schema.Athlete))
#dados.add((Schema.Athlete, RDFS.subClassOf, Schema.Person))

In [ ]:
# Query 2 - Todos os jogadores que têm uma posição ofensiva

from SPARQLWrapper import SPARQLWrapper, XML, TSV, POST, DIGEST

sparql = SPARQLWrapper(("http://localhost:3030/dataset/query")

query= """
prefix NFL: <http://www.nfl.com/>
prefix Schema: <http://schema.org/>
prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#>
prefix xsd: <http://www.w3.org/2001/XMLSchema#>

SELECT ?s ?p ?o 
WHERE {
  ?s ?p NFL:offensivePlayer .
} 
"""
sparql.setQuery(query)
results = sparql.query()
g=results.convert()

print(g.serialize(format='ttl').decode("utf-8"))

In [ ]:
# Query 3 - Todos os jogadores que têm uma posição defensiva

sparql = SPARQLWrapper(("http://localhost:3030/dataset/query")

query= """
prefix NFL: <http://www.nfl.com/>
prefix Schema: <http://schema.org/>
prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#>
prefix xsd: <http://www.w3.org/2001/XMLSchema#>

SELECT ?s ?p ?o 
WHERE {
  ?s ?p NFL:defensivePlayer .
} 
"""
sparql.setQuery(query)
results = sparql.query()
g=results.convert()

print(g.serialize(format='ttl').decode("utf-8"))